## Some data exploration

In [1]:
# import statements 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model  import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_predict
import datetime as dt

quick look at the Sambanis dataset

In [2]:
sambanis = pd.read_csv("SambnisImp.csv",index_col=0)
print(f'Sambanis dataset has {sambanis.shape[0]} rows with {sambanis.shape[1]} features.')
#civil_war contains only the observations with a civil war
civil_war = sambanis[sambanis["warstds"]==1]
print(f'There are {civil_war.shape[0]} civil war in the dataset.')
nb_country_cw = len(civil_war['cowcode'].value_counts())
print(f'There are {nb_country_cw} different country which had a CW (some countries had multiple CW)')
print(f'The data ranges from 1945 to 2000. The first CW dates form {min(civil_war.year)} and the last {max(civil_war.year)}')

Sambanis dataset has 7140 rows with 285 features.
There are 116 civil war in the dataset.
There are 72 different country which had a CW (some countries had multiple CW)
The data ranges from 1945 to 2000. The first CW dates form 1946 and the last 1999


quick look at trade_reg.csv. created from Trade-Register-1950-2000.txt

In [3]:
trade = pd.read_csv("trade_reg.csv",sep='\t')

In [4]:
print(f'The data ranges from {min(trade.year_order)} to {max(trade.year_order)}.')
trade.supplier.value_counts()  

The data ranges from (1940) to 2000.


United States          6226
Soviet Union           3358
France                 1854
United Kingdom         1689
Germany                 706
                       ... 
Bangladesh                1
FMLN (El Salvador)*       1
Guatemala                 1
Sri Lanka                 1
Cyprus                    1
Name: supplier, Length: 115, dtype: int64

In [5]:
trade.recipient.value_counts()

Egypt                  433
India                  427
Turkey                 409
Greece                 407
Thailand               382
                      ... 
MPLA (Portugal)*         1
 Congo                   1
Russia                   1
 Zimbabwe                1
 East Germany (GDR)      1
Name: recipient, Length: 298, dtype: int64

Adding the "country" column in sambanis dataset based on the cowcodes

In [6]:
#loading the cowcode dataset
cowcodes = pd.read_csv("COW_country_codes.csv")
cowcodes = cowcodes.rename(columns={'CCode': 'cowcode', 'StateNme':'country'})
#create the dictionaries
dict_cowcode = dict(zip(cowcodes.cowcode,cowcodes.country))
dict_cowcode_abb = dict(zip(cowcodes.cowcode,cowcodes.StateAbb))

sambanis['country'] = sambanis.cowcode.map(dict_cowcode)
sambanis['country_abb'] =sambanis.cowcode.map(dict_cowcode_abb) 
#don't know if it is necessary to add the abbreviation but I figured it could be useful

Counting how many countries who had a civil war were also the recipient of weapons within a time range

In [7]:
trade.year_order = trade.year_order.str.strip('( )') 
#j'ai juste enlevé les parenthèse mais je sais pas si on devrait garder ces lignes ou pas
#trade.recipient = trade.recipient.str.strip(' ')

#transform to datetime
sambanis['year'] = pd.to_datetime(sambanis['year'],format='%Y')
trade['year_order'] = pd.to_datetime(trade['year_order'],format='%Y')

In [8]:
civil_war = sambanis[sambanis["warstds"]==1]
civil_war['import_weapons'] = 0

C:\Users\Cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
 for id in civil_war.index:
    country = civil_war.country.loc[id]
    date = civil_war.year.loc[id]
    start = date + dt.timedelta(days=-365) # 3 years, to be changed?
    end = date + dt.timedelta(days=365)
    mask = (trade['year_order'] >= start) & (trade['year_order'] <= end)
    trade_tmp = trade.loc[mask]
    if (trade_tmp.recipient.str.contains(country).sum()>0):
        civil_war.loc[id,'import_weapons'] = 1

C:\Users\Cindy\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
civil_war['import_weapons'].sum() 

82